In [ ]:
%pip install playwright databricks-sdk
%restart_python

In [ ]:
import os

from databricks.sdk import WorkspaceClient


def strip_scheme(host_or_url: str) -> str:
    return host_or_url.replace("https://", "").replace("http://", "").strip("/")


dbutils.widgets.text("app_url", "<app-name> OR <app-name>-<your-databricks-instance>.azuredatabricks.net")
dbutils.widgets.text("databricks_host", os.environ.get("DATABRICKS_HOST", ""))
dbutils.widgets.text("scope", "")
dbutils.widgets.text("expected_header_text", "Hello World")

raw_app_url = dbutils.widgets.get("app_url").strip()
raw_databricks_host = dbutils.widgets.get("databricks_host").strip()
scope = dbutils.widgets.get("scope").strip()
expected_header_text = dbutils.widgets.get("expected_header_text").strip()

client_id = dbutils.secrets.get(scope=scope, key="client_id")
client_secret = dbutils.secrets.get(scope=scope, key="client_secret")

workspace_kwargs = {
    "client_id": client_id,
    "client_secret": client_secret,
}
if raw_databricks_host:
    workspace_kwargs["host"] = raw_databricks_host

workspace = WorkspaceClient(**workspace_kwargs)

auth_headers = workspace.config.authenticate()
workspace_host = strip_scheme(workspace.config.host or raw_databricks_host or os.environ.get("DATABRICKS_HOST", ""))

app_host = strip_scheme(raw_app_url)
if not app_host:
    raise ValueError("app_url is required. Pass either app name or full app host.")

# If app_url is just the app name, compose the Databricks Apps host.
if "." not in app_host:
    if not workspace_host:
        raise ValueError(
            "Unable to resolve workspace host. Set databricks_host or DATABRICKS_HOST "
            "when app_url is only the app name."
        )
    app_host = f"{app_host}-{workspace_host}"

url = f"https://{app_host}".rstrip("/")

items = workspace.workspace.list(path="/")
print("Authentication successful. Workspace root contents:")
for item in items:
    print(f"- {item.path} ({item.object_type})")

browser_headers = {}
if "Authorization" in auth_headers:
    browser_headers["Authorization"] = auth_headers["Authorization"]



In [ ]:
from playwright.async_api import Error as PlaywrightError
from playwright.async_api import async_playwright

async def run_playwright_test():
    async with async_playwright() as p:
        try:
            browser = await p.chromium.launch()
        except PlaywrightError as exc:
            raise RuntimeError(f"Chromium launch failed: {exc}") from exc
        try:
            context = await browser.new_context(extra_http_headers=browser_headers)
            page = await context.new_page()
            response = await page.goto(url, wait_until="domcontentloaded")

            if response is None:
                raise RuntimeError("Navigation completed without an HTTP response.")

            if response.status in {401, 403}:
                raise AssertionError(
                    f"Authentication failed with HTTP {response.status}. "
                    "Check databricks_host, scope, and client secrets."
                )

            text_example = await page.locator("h1").text_content()
            if text_example is None:
                raise AssertionError("Expected an <h1> element but none was found.")

            text_example = text_example.strip()
            assert text_example == expected_header_text, (
                f"Expected h1={expected_header_text!r}, got {text_example!r}"
            )
        finally:
            await browser.close()

await run_playwright_test()
print("Test passed!")
